In [1]:
import copy
import csv
import os
import subprocess
import time
from time import sleep
import timeit

In [2]:
from sqlalchemy import create_engine
import datetime
import pandas as pd
import numpy as np
from time import strftime

In [3]:
engine = create_engine('sqlite:///foo.db')

In [4]:
class Timer:
    def __init__(self, func=timeit.default_timer):
        self.elapsed = 0.0
        self._func = func
        self._start = None
    def start(self):
        if self._start is not None:
            raise RuntimeError('Already started')
        self._start = self._func()
    def stop(self):
        if self._start is None:
            raise RuntimeError('Not started')
        end = self._func()
        self.elapsed += end - self._start
        self._start = None
    def reset(self):
        self.elapsed = 0.0
    @property
    def running(self):
        return self._start is not None
    def __enter__(self):
        self.start()
        return self
    def __exit__(self, *args):
        self.stop()


In [5]:
def ping_return_code(hostname):
    """Use the ping utility to reach the host. Send 1 packet
    ('-c 1'), exit successfully after receiving one reply packet (return code = 0) or a failed return code (return code = 1)
    if no ping response is received.
    """
    ret_code = subprocess.call(['ping', '-n', '1', hostname],
                            stdout=open(os.devnull, 'w'),
                            stderr=open(os.devnull, 'w'))
    if ret_code != 0:
        ret_code = hostname
        print ret_code
#
    return ret_code

def timer_state(host_defect, defect_timer):
    testList_ = 0
    if host_defect == testList_:
        if defect_timer == False:
            return "not running", defect_timer  
        else:
            print 'stop'
            defect_timer = False
            return "stop", defect_timer
    else:
        if defect_timer == True:
            return "running", defect_timer
        else:
            print 'start'
            defect_timer = True
            return "start", defect_timer
#

def verify_availability(host_list):
    """For each hostname in the list, attempt to reach it using ping. If any ping
    return code is not zero then network is unavailable. Assumes that the hostnames
    are valid.
    """
    return_code = ping_return_code(host_list)   
    return return_code
#

In [6]:
def SQL_write(table, badHost, timeIt):
    timeNow = strftime("%Y-%m-%d %H:%M:%S")
    storeDict = {'Date': timeNow, 'Bad Hosts': badHost, 'Elapsed Time': timeIt}
    df = pd.DataFrame([storeDict])
    df.to_sql(table, engine, if_exists = 'append', index = False)

In [15]:
"""
This iterates over a single host 'repeat_times' and
then continues to the next host in the hostlist.csv file.
Count_loop is the number of times the complete set of hosts
is iterated over 'repeat_times'.
"""
with open('hostlist.csv', 'rb') as csvfile:
    names = csv.reader(csvfile)
    for row in names: hostlist_ = [i for i in row]
print hostlist_
#
# Set the SQL table data will be written to.
table = 'foo-Itron'
# Timestamp the beginning of this trial with a fictitious record.
SQL_write(table, '0.0.0.0', 0.0)
#
# Init the program's parameters.
repeat_times = 1     
defect_timer = False
state = 'not running'
stop_me = 'no'
count_loop = 1
#
while count_loop > 0:
    hostlist = [i for i in hostlist_]
    for host_ in hostlist:
        print host_
        if (state == 'running') or (state == 'start'):
            t.stop()
            state = 'not running'
            defect_timer = False
            Elapsed = str(t.elapsed)
            print "elapsed = ", t.elapsed
            t.reset()
#
            SQL_write (table, bad_host, Elapsed)
        for repeat in range(repeat_times):
            host_defect = verify_availability(host_)
            sleep(1)
            state, defect_timer = timer_state(host_defect, defect_timer)
            if state == 'start':
                t = Timer()
                t.start()
                bad_host = host_defect
            elif state == 'stop':
                t.stop()
                Elapsed = str(t.elapsed)
                t.reset()
#
                SQL_write (table, bad_host, Elapsed)
            else:
                try:
                    f = open('continue_running', 'r')
                    f.close()
                except:
                    print "except"
                    if (state == 'running') or (state == 'start'):
                        t.stop()
                        state = 'not running'
                        defect_timer = False
                        Elapsed = str(t.elapsed)
                        print "elapsed = ", t.elapsed
                        t.reset()
#
                        SQL_write (table, bad_host, Elapsed)
#
                    print "BREAK"
#
                    SQL_write (table, '255.255.255.255', 0.0)
                    stop_me = 'yes'
                    break
#
        if stop_me == 'yes': break
    if (state == 'running') or (state == 'start'):
        t.stop()
        state = 'not running'
        defect_timer = False
        Elapsed = str(t.elapsed)
        print "elapsed = ", t.elapsed
        t.reset()
#
        SQL_write (table, bad_host, Elapsed)
#
    SQL_write (table, '255.255.255.255', 0.0)
    count_loop -= 1

['192.168.95.75', '192.168.95.76', '192.168.95.77', '192.168.95.78', '192.168.95.79', '192.168.95.80', '192.168.95.81', '192.168.95.82', '192.168.95.83', '192.168.95.84', '192.168.95.85', '192.168.95.86']
192.168.95.75
192.168.95.75
start
192.168.95.76
elapsed =  0.00038273020732
192.168.95.76
start
192.168.95.77
elapsed =  0.000371066713683
192.168.95.77
start
192.168.95.78
elapsed =  0.000936012817306
192.168.95.78
start
192.168.95.79
elapsed =  0.000473733393505
192.168.95.79
start
192.168.95.80
elapsed =  0.000177955578124
192.168.95.80
start
192.168.95.81
elapsed =  0.000878952492485
192.168.95.81
start
192.168.95.82
elapsed =  0.000505511175334
192.168.95.82
start
192.168.95.83
elapsed =  0.000325320676325
192.168.95.83
start
192.168.95.84
elapsed =  0.000816654071969
192.168.95.84
start
192.168.95.85
elapsed =  0.000372463539293
192.168.95.85
start
192.168.95.86
elapsed =  0.000372742904574
192.168.95.86
start
elapsed =  1.30603191337e-05


In [18]:
df = pd.read_sql_table('foo-Itron', engine, columns = ['Date', 'Bad Hosts', 'Elapsed Time'])

In [19]:
df

,Date,Bad Hosts,Elapsed Time
0,2017-11-28 19:15:12,0.0.0.0,0.000000
1,2017-11-28 19:15:37,192.168.95.75,19.982652
2,2017-11-28 19:16:02,192.168.95.76,20.034561
3,2017-11-28 19:16:27,192.168.95.77,19.996309
4,2017-11-28 19:16:52,192.168.95.78,20.000595
5,2017-11-28 19:17:17,192.168.95.79,19.997404
6,2017-11-28 19:17:42,192.168.95.80,20.001712
7,2017-11-28 19:18:07,192.168.95.81,19.998469
8,2017-11-28 19:18:32,192.168.95.82,19.989453
9,2017-11-28 19:18:57,192.168.95.83,20.000435


In [ ]:
"""
The following cells cleans up the foo.db table for Cisco/ABB.
It is throw away and not of much value, except as a 
trail for processing that has been done.
"""

In [17]:
df_all = df[29:]

In [18]:
df_all.index = range(0,len(df_all))

In [19]:
df_all.describe()

,Elapsed Time
count,125.000000
mean,1.537987
std,1.811823
min,0.000000
25%,1.064953
50%,1.076116
75%,1.089929
max,11.106219


In [21]:
node = '192.168.95.74'
df_all[df_all['Bad Hosts'] == node].shape[0]

13

In [22]:
df_all

,Date,Bad Hosts,Elapsed Time
0,2017-11-27 08:29:42,0.0.0.0,0.000000
1,2017-11-27 08:29:43,255.255.255.255,0.000000
2,2017-11-27 08:29:43,255.255.255.255,0.000000
3,2017-11-27 08:30:11,0.0.0.0,0.000000
4,2017-11-27 08:30:27,192.168.95.64,1.110289
5,2017-11-27 08:31:25,192.168.95.74,1.068948
6,2017-11-27 08:31:29,255.255.255.255,0.000000
7,2017-11-27 08:35:31,0.0.0.0,0.000000
8,2017-11-27 08:47:00,192.168.95.63,1.069215
9,2017-11-27 08:48:09,192.168.95.63,1.079745
